In [ ]:
%matplotlib inline
import pandas as pd
import nupis
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import geopandas as gp
import pysal as ps
import pysal.contrib.viz.mapping as maps
import palettable

pd.options.mode.chained_assignment = None

sns.set(style='whitegrid', palette='bright', context='notebook')

### Setores censitários

Para baixar as malhas dos outros estados do Brasil ir em:

ftp://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2010/setores_censitarios_shp/

Para baixar o arquivo com as variáveis por setores censitários em MG:

ftp://ftp.ibge.gov.br/Censos/Censo_Demografico_2010/Resultados_do_Universo/Agregados_por_Setores_Censitarios/

Dicas em: http://www.analisededadosempython.org/espacial

In [ ]:
# estado de Minas Gerais dividido em setores censitários
setores = gp.read_file('31SEE250GC_SIR.shp') 

In [ ]:
# a partir do shapefile de setores de MG, selecionar o município de Varginha e apenas os setores urbanos
vga = setores.query('NM_MUNICIP == "VARGINHA"')
mapa_vga = vga.query('TIPO == "URBANO"')
mapa_vga.plot(figsize=(9,9)).set_axis_off();

### Agora vamos pegar variáveis sobre os setores censitários de MG e utilizar os de Varginha.

O arquivo que usaremos será o 'basico_MG.csv'

Obs.: peguei esse arquivo em: 

ftp://ftp.ibge.gov.br/Censos/Censo_Demografico_2010/Resultados_do_Universo/Agregados_por_Setores_Censitarios/).

As variáveis desse arquivo estão codificadas, para saber o que significam, consultar o arquivo 'base_setores_censitarios_censo_2010.pdf'

In [ ]:
# carregar arquivo com dados sobre os setores censitários de MG
mg = pd.read_csv('basico_MG.csv', encoding='latin1', sep=';', decimal=',')
# salvar as informações de Varginha num dataframe
vga_df = mg.query('Nome_do_municipio == "VARGINHA"')
vga_df = vga_df.query('Situacao_setor == [1, 2, 3]')   # só setores urbanos
# o objeto 'mapa_vga' contém as informações geográficas de todos os setores censitários de Varginha
# vamos incluir as informações das variáveis presentes em 'vgn_df' neste dataframe
# e chamá-lo de geodf 
# como usaremos a coluna de código do setor censitário como a variável para unir os dois conjuntos de
# dados, o nome dessa coluna deve ser o mesmo nos dois (chamarei de 'Cod_setor')
df = vga_df
geodf = mapa_vga
# geodf.geometry = geodf.geometry.simplify(0.001)
geodf.rename(columns={'CD_GEOCODI': 'CD_setor'}, inplace=True)
# os códigos não estão sendo lidos como inteiros, transformá-los em inteiros
geodf.CD_setor = geodf.CD_setor.astype('int64')
# para que os dois dataframes tenham a coluna de códigos com o mesmo nome,
# alterar para CD_setor
df.rename(columns={'Cod_setor': 'CD_setor'}, inplace=True)

In [ ]:
# unir o dataframe de informações geográficas com o outro dataframe com as variáveis
# Obs.: 'suffixes' é para o caso de haver variáveis de mesmo nome, incluir alguma coisa no final do nome
# em cada dataframe
varginha = pd.merge(geodf, df, on='CD_setor')
varginha

In [ ]:
varginha.columns

### Tarefa: escolher uma das variáveis (V001, $\dots$, V012) e realizar a AEDE (análise exploratória de dados espaciais)

- plotar mapa temático com intervalos iguais  
- plotar mapa temático com intervalos baseados em quantis (melhorar a posição da legenda se necessário)  
- obter boxplot da variável  
- obter $I$ de Moran para a variável e avaliar significância  
- obter diagrama de dispersão de Moran para a variável (com nomes nos eixos)  
- obter mapa LISA
- plotar os bairros correspondentes a *clusters* ou *outliers*

V007: valor do rendimento nominal médio mensal das pessoas responsáveis por domicílios particulares permanentes (com
rendimento)

In [ ]:
# aqui substituir por outra variável
v = 'V007'

In [ ]:
# obter a matriz w de pesos espaciais
# lembrar de salvar um shapefile (.shp) apenas com as informações dos setores de Varginha (objeto mapa_vga)
# como fizemos na aula p4
mapa_vga.to_file('vga_setores.shp')    # salvar o shapefile dos setores
w = ps.queen_from_shapefile('vga_setores.shp') # calcular a matriz de vizinhança

In [ ]:
# mapa temático
ax = varginha.plot(column=v, scheme='equal_interval', k=4, linewidth=0, 
                   figsize=(7,7), legend=True, cmap='Set1')
leg = ax.get_legend()
leg.set_bbox_to_anchor((1.5, 0.7)); # alterar valores de x e y: posição
ax.set_axis_off();

In [ ]:
# mapa temático - problema devido aos valores faltantes da variável V007
ax = varginha.plot(column=v, scheme='quantiles', k=4, linewidth=0, 
                   figsize=(7,7), legend=True, cmap='Set1')
leg = ax.get_legend()
leg.set_bbox_to_anchor((1.5, 0.7)); # alterar valores de x e y: posição
ax.set_axis_off();

In [ ]:
# usar algum gráfico descritivo que represente os mesmos valores e dê ideia da dispersão
sns.boxplot(v, data=varginha, orient='v');

In [ ]:
IM = ps.Moran(varginha[v], w)  # variável no dataframe e matriz de vizinhança 
nupis.moran_resumo(IM)         # função do nupis que retorna uma saída organizada do I de Moran

**Problema**: houve alguma divisão por zero, provavelmente há algum valor nulo da variável escolhida

In [ ]:
varginha[v]

In [ ]:
# retornar setores com valor da variável nulo
varginha[varginha[v].isnull()]

In [ ]:
# substituir os valores faltantes pela média da variável
varginha[v] = varginha[v].fillna(varginha[v].mean())

In [ ]:
varginha[v]

In [ ]:
# mapa temático em intervalos de valores iguais
ax = varginha.plot(column=v, scheme='equal_interval', k=4, linewidth=0, 
                   figsize=(7,7), legend=True, cmap='Set1')
leg = ax.get_legend()
leg.set_bbox_to_anchor((1.5, 0.7)); # alterar valores de x e y: posição
ax.set_axis_off();

Analisar o mapa temático dividido em intervalos de valores iguais:



In [ ]:
# refazendo o mapa temático de quantis que havia dado erro por causa dos valores faltantes
ax = varginha.plot(column=v, scheme='quantiles', k=4, linewidth=0, 
                   figsize=(7,7), legend=True, cmap='Set1')
leg = ax.get_legend()
leg.set_bbox_to_anchor((1.5, 0.7)); # alterar valores de x e y: posição
ax.set_axis_off();

In [ ]:
IM = ps.Moran(varginha[v], w)  # variável no dataframe e matriz de vizinhança 
nupis.moran_resumo(IM)           # função do nupis que retorna uma saída organizada do I de Moran

In [ ]:
# a partir daqui, o que devemos fazer se quisermos tirar as ilhas

## ilhas
# w.islands
## tirar a ilha do objeto mapa_vga (que contém o shapefile) e do objeto varginha (o dataframe completo)
# mapa_vga.drop(mapa_vga.index[w.islands], inplace=True)
# varginha.drop(varginha.index[w.islands], inplace=True)
# varginha.shape
## plotar apenas o mapa com os limites territoriais
# mapa_vga.plot();
## plotar o mapa sem grades e eixos
# mapa_vga.plot().set_axis_off();
## salvar o shapefile e a matriz w novamente (sem a ilha)
# mapa_vga.to_file('vga_setores.shp')    # salvar o shapefile dos setores
# w = ps.queen_from_shapefile('vga_setores.shp') # calcular a matriz de vizinhança

In [ ]:
nupis.moran_dispersao(IM)   # diagrama de dispersão de Moran que recebe o objeto criado antes

In [ ]:
nupis.moran_dispersao(IM, xlabel='renda mensal média', ylabel='renda mensal média nos vizinhos')

### Mapas LISA

In [ ]:
nupis.lisa_mapa(varginha[v], 'vga_setores.shp', p_thres=0.05)   # função do nupis para obter o mapa LISA

In [ ]:
shapefile = 'vga_setores.shp'
w = ps.queen_from_shapefile(shapefile)
lisa = ps.Moran_Local(varginha[v], w)
p_thres = 0.05

In [ ]:
import numpy as np
lisa.p_sim  # pseudovalores do LISA
sig = lisa.p_sim < 0.05  # identificar significativos
lisa.p_sim[sig]
posicoes = np.where(sig)
varginha['quad'] = lisa.q
mun_sig = varginha.loc[posicoes[0], ['NM_BAIRRO', 'quad']]
# ou ssm.loc[posicoes[0], ['nome_mun', 'quad']].query('quad == 4')
cidades_escolhidas = varginha.iloc[mun_sig.index, :]
cidades_escolhidas

In [ ]:
# plotar os nomes dos clusters e outliers espaciais 
fig = plt.figure(figsize=(15, 15))
shp = ps.open(shapefile)
base = maps.map_poly_shp(shp)
base = maps.base_lisa_cluster(base, lisa, p_thres=p_thres)
base.set_edgecolor('1')
base.set_linewidth(0.1)
ax = maps.setup_ax([base], [shp.bbox])

boxes, labels = maps.lisa_legend_components(lisa, p_thres=p_thres)
plt.legend(boxes, labels, fancybox=True)

for i in cidades_escolhidas.index:

    plt.text(cidades_escolhidas.geometry.centroid[i].coords[0][0], cidades_escolhidas.geometry.centroid[i].coords[0][1], 
             cidades_escolhidas.NM_BAIRRO[i],
             fontsize=10, horizontalalignment='center', verticalalignment='bottom')